In [237]:
#how does word2vec work for query expansion and for synonyms?
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.summarization.textcleaner import get_sentences#clean_text_by_sentences

import pandas as pd
import numpy as np

from tqdm import tqdm
import xml.etree.ElementTree as Et
from collections import defaultdict

from sklearn.model_selection import KFold
from sklearn.metrics import ndcg_score

import os
import re
import string

In [238]:
from pyserini.search import pysearch
from pyserini.search import pyquerybuilder
from pyserini.pyclass import autoclass
from pyserini.analysis.pyanalysis import get_lucene_analyzer
from pyserini.index import pyutils
from pyserini.analysis import pyanalysis

index_loc = '/home/tmschoegje/Desktop/caos-19/lucene-index-cord19-paragraph-2020-05-19/'
searcher = pysearch.SimpleSearcher(index_loc)
index_utils = pyutils.IndexReaderUtils(index_loc)

In [239]:
#train model on paper abstracts

#load metadata.csv
def prepTREC(fname):
    #get valid TREC ids for this round
    TRECids = []
    f = open(fname)
    for line in f.readlines():
        if line[-1] == '\n':
            line = line[:-1]
        TRECids.append(line)
    f.close()
    
    metadata = pd.read_csv("/home/tmschoegje/Desktop/caos-19/metadata.csv")
    #now we filter all TREC ids we don't need
    #print(np.array(metadata).shape)
    #print(np.array(TRECids).shape)
######    metadata = metadata[metadata.cord_uid.isin(TRECids)]
    #there's 33 duplicates (same paper from multiple sources)
    metadata.drop_duplicates(subset='cord_uid', keep='first', inplace=True)
    
    return metadata

def prep_w2v(fname, data):
    #Check if a model exists
    if(os.path.isfile(fname)):
        print("Loaded word2vec model " + fname)
        word_vectors = KeyedVectors.load(fname)
    else:
        print("Training word2vec model " + fname)
        
        #Remove items with bad abstracts
        docs = data[~data.abstract.isin(["Unknown", "unknown", ""])]
        #remove items with mising abstracts
        docs = data[~data.abstract.isnull()]
        metadatana = docs
        docvals = docs['abstract']
        
        #join all docs into a large doc
        #docvals = " . ".join( docvals.values.tolist())
        docs = docvals.values.tolist()
        
        tokenized = []
        for doc in docs:
            sentences = get_sentences(doc)
            for sent in sentences:
                tokenized.append(sent.translate(str.maketrans('', '', string.punctuation)).split())
        
        model = Word2Vec(tokenized, size=100, window=5, min_count=5, workers=4)
        word_vectors = model.wv
        word_vectors.save(fname)
        
    print('ready')
    return word_vectors
        
word_vectors = prep_w2v("../word2vec7.model", prepTREC('../trecdata/docids-rnd3.txt'))

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Loaded word2vec model ../word2vec7.model
ready


In [240]:
#qexp based on short task descriptions
#print(model.wv)
#word_vectors.similar_by_word("genetics")
similars = word_vectors.most_similar(positive=['coverage'])
print(similars)
print()
print([w[0] for w in similars][:5])

#optional extension: https://radimrehurek.com/gensim/models/phrases.html#module-gensim.models.phrases

[('accessibility', 0.5386149883270264), ('acceptance', 0.5321283340454102), ('insurance', 0.5319286584854126), ('price', 0.5314955711364746), ('incomes', 0.5182398557662964), ('campaigns', 0.5108014941215515), ('compliance', 0.5098401308059692), ('cost', 0.5023601055145264), ('density', 0.4933677315711975), ('poverty', 0.4841878414154053)]

['accessibility', 'acceptance', 'insurance', 'price', 'incomes']


In [241]:
#qexp based on query terms in topic (i think this idea came from faegheh)
test="hello this is a document. two sentences here."
sentences = get_sentences(test)
input = []
for sent in sentences:
    print(sent.translate(str.maketrans('', '', string.punctuation)).split())

['hello', 'this', 'is', 'a', 'document']
['two', 'sentences', 'here']


In [262]:
# So let's see how this works for expansion

stops = {'of', 'to', 'the', 'for', 'in', 'on', 'and',
         'is', 'will', 'Is', 'or', 'are', 'there', 'that', 'an', 'with',
         'at', 'by', 'but', 'Are', 'be', 'this', 'if', 'they?', 'they',
         "a", "an", "and", "are", "as", "at", "be", "but", "by",
"for", "if", "in", "into", "is", "it",
"no", "not", "of", "on", "or", "such",
"that", "the", "their", "then", "there", "these",
"they", "this", "to", "was", "will", "with", "The"}

#where do we base our terms in this expansion?
task_version = "tfidf"
#task_version = "qexp"

#query_version = "query"
query_version = "qexp"

#return n similar terms to term
def term_expansion(term, n):
    if type(term) == str:
        if term in word_vectors.vocab:
            similars = word_vectors.most_similar(positive=[term])
            return ([w[0] for w in similars][:n])
    return []

def build_query(bm25, nars, task, bm25_split, nars_split, task_split, task_confidence, task_id):
    if(query_version == "query"):
        bm25 = [b for b in bm25 if b.lower() not in stops]
    elif(query_version == "qexp"):
        newterms = []
        for term in bm25:
            newterms.extend(term_expansion(term, 3))
        bm25.extend(newterms)
        bm25 = [b for b in bm25 if b.lower() not in stops]
    
    if(task_version == "tfidf"):
        task = [t for t in task if t not in stops]
    elif(task_version == "qexp"):
        #expand based on doc2vec - add 3 terms
        newterms = []
        for term in list_of_tasks_short[task_id].split():
            newterms.extend(term_expansion(term, 3))
        task = list_of_tasks_short[task_id].split()
        
        task.extend(newterms)
        task = [t for t in task if t.lower() not in stops]
    nars = [n for n in nars if n.lower() not in stops]
    
    builder = pyquerybuilder.get_boolean_query_builder()
    
    for b in bm25:
        b = pyquerybuilder.get_term_query(b)
        boostquery = pyquerybuilder.get_boost_query(b, bm25_split/len(bm25))
        builder.add(boostquery, pyquerybuilder.JBooleanClauseOccur['should'].value)

    #using terms from kaggle tasks
    #if(task_confidence > 0.6):
    for t in task:
        t = pyquerybuilder.get_term_query(t)
        boostquery = pyquerybuilder.get_boost_query(t, task_split/len(task) )#* task_confidence
        builder.add(boostquery, pyquerybuilder.JBooleanClauseOccur['should'].value)
    #print(task)
    #using terms from qrel docs
    #if(task_confidence > 0.8):
    #    for t, boost in task.items():
    #        t = pyquerybuilder.get_term_query(t)
    #        boostquery = pyquerybuilder.get_boost_query(t, task_split/boost)
    #        builder.add(boostquery, pyquerybuilder.JBooleanClauseOccur['should'].value)
        
    for n in nars:
        try:
            n = pyquerybuilder.get_term_query(n)
        except:
            print(n)
            continue
        boostquery = pyquerybuilder.get_boost_query(n, nars_split/len(nars))
        builder.add(boostquery, pyquerybuilder.JBooleanClauseOccur['should'].value)
        
    return builder.build()

In [263]:
#some constants

tasks = [
    ['environmental transmission', 'incubation', 'contagious', 'persistence', 'stability', 'physical', 'weather',
     'epidemiology', 'shedding', 'reproductive number', 'modes of transmission', 'virulent', 'asymptomatic', 'pathogen',
     'evolutionary host', 'transmission host'],
    ['smoking', 'risk', 'pulmonary', 'pre-condition', 'co-infection', 'high-risk', 'severe', 'susceptible', 'fatality',
     'neonates', 'respitory', 'condition', 'pre-existing', 'pregnant', 'morbidities'],
    ['human-animal', 'origin', 'genetics', 'evolution', 'genome', 'sample sets', 'genomic', 'strain', 'livestock',
     'animal host', 'natural history', 'genetic drift', 'mutation', 'genomics', 'sequencing'],
    ['vaccine', 'therapeutic', 'treat', 'drugs', 'pharmaceuticals', 'recipients', 'ADE', 'complication', 'antiviral',
     'prophylaxis', 'cloroquine', 'vaccination', 'immume respone'],
    ['medical care', 'surge capacity', 'nursing home', 'allocation', 'personal protective equirement',
     'clinical characterization', 'nursing', 'care', 'Extracorporeal membrane oxygenation', 'ECMO',
     'mechanical ventilation', 'extrapulmonary manifestations', 'cardiomyopathy', 'cardiac arrest',
     'regulatory standards', 'N95 masks', 'elastomeric respirators', 'telemedicine', 'steroids', 'high flow oxygen',
     'supportive interventions'],
    ['NPI', 'non-pharmaceutical intervention', 'school closure', 'travel ban', 'quarantine', 'mass gathering',
     'social distancing', 'public health advice', 'economic impact'],
    ['counties', 'geographic', 'geography', 'mortality rate', 'spread', 'mutations'],
    ['diagnostic', 'surveillance', 'detection', 'screening', 'ELISAs', 'capacity', 'testing', 'point-of-care',
     'rapid testing', 'pathogen', 'reagent', 'cytokines', 'response markers', 'swabs'],
    ['ethical', 'social science', 'principles', 'standards', 'ethics', 'psychological health', 'fear', 'anxiety',
     'stigma', 'sociology'],
    ['collaboration', 'nomenclature', 'data standards', 'information sharing', 'communication', 'collaborate',
     'coordination', 'misunderstanding', 'action plan']
]

#classification with confidence
rnd3classes = [2, 0, 3, 0, 3, 7, 7, 7, 6, 5, 4, 5, 0, 0, 0, 0, 4, 5, 5, 1, 0, 1, 1, 1, 1, 7, 7, 3, 3, 3, 2, 2, 3, 3, 9, 2, 2, 3, 3, 2]
rnd3confidence = [1, 1, 1, 0.5, 0.5, 1, 1, 0.5, 0.5, 1, 0, 0.5, 1, 0, 1, 1, 0.75, 0.25, 0.25, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 1, 0.5, 1, 1, 1, 0, 1, 1, 0, 0, 0.5, 1, 0, 0, 1]


#adapted from constants.py
#has the contstants we use for doc2vec reranking

#these were defined before
#rnd1classes = [2,0,3,0,3,6,3,6,7,5,4,5,0,0,0,0,6,5,5,1,0,0,1,1,1,6,6,3,3,3]
#rnd3classes = [2,0,3,0,3,7,7,7,6,5,4,5,0,0,0,0,4,5,5,1,0,1,1,1,1,7,7,3,3,3,, 2, 3, 3, 9, 2, 2, 3, 3, 2]
#rnd2classes = [2,0,3,0,3,6,3,6,7,5,4,5,0,0,0,0,3,5,5,1,0,0,1,1,6,6,6,3,3,3,2,2,3,1,9]
#rnd3confidence = [1, 1, 1, 0.5, 0.5, 1, 1, 0.5, 0.5, 1, 0, 0.5, 1, 0, 1, 1, 0.75, 0.25, 0.25, 0.5, 0.5, 0.75, 0.75, 0.75, 0.75, 1, 0.5, 1, 1, 1, 0, 1, 1, 0, 0, 0.5, 1, 0, 0, 1]

task_1_short = "transmission incubation environment"
task_2_short = "risk factors"
task_3_short = "genetics origin evolution"
task_4_short = "vaccines therapeutics"
task_5_short = "medical care"
task_6_short = "effectiveness non pharmaceutical interventions"
task_7_short = "diagnostics surveillance"
task_8_short = "geographic spread"
task_9_short = "ethical social considerations"
task_10_short = "information sharing collaboration"

task_1 = "What is known about transmission, incubation, and environmental stability? What do we know about natural history, transmission, and diagnostics for the virus? What have we learned about infection prevention and control? Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery. Prevalence of asymptomatic shedding and transmission (e.g., particularly children). Seasonality of transmission. Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic/phobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding). Persistence and stability on a multitude of substrates and sources (e.g., nasal discharge, sputum, urine, fecal matter, blood). Persistence of virus on surfaces of different materials (e,g., copper, stainless steel, plastic). Natural history of the virus and shedding of it from an infected person. Implementation of diagnostics and products to improve clinical processes. Disease models, including animal models for infection, disease and transmission. Tools and studies to monitor phenotypic change and potential adaptation of the virus. Immune response and immunity. Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings. Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings. Role of the environment in transmission"
task_2 = "What do we know about COVID-19 risk factors? What have we learned from epidemiological studies? Data on potential risks factors. Smoking, pre-existing pulmonary disease. Co-infections (determine whether co-existing respiratory/viral infections make the virus more transmissible or virulent) and other co-morbidities. Neonates and pregnant women. Socio-economic and behavioral factors to understand the economic impact of the virus and whether there were differences. Transmission dynamics of the virus, including the basic reproductive number, incubation period, serial interval, modes of transmission and environmental factors. Severity of disease, including risk of fatality among symptomatic hospitalized patients, and high-risk patient groups. Susceptibility of populations. Public health mitigation measures that could be effective for control."
task_3 = "What do we know about virus genetics, origin, and evolution? What do we know about the virus origin and management measures at the human-animal interface? Real-time tracking of whole genomes and a mechanism for coordinating the rapid dissemination of that information to inform the development of diagnostics and therapeutics and to track variations of the virus over time. Access to geographic and temporal diverse sample sets to understand geographic distribution and genomic differences, and determine whether there is more than one strain in circulation. Multi-lateral agreements such as the Nagoya Protocol could be leveraged. Evidence that livestock could be infected (e.g., field surveillance, genetic sequencing, receptor binding) and serve as a reservoir after the epidemic appears to be over. Evidence of whether farmers are infected, and whether farmers could have played a role in the origin. Surveillance of mixed wildlife- livestock farms for SARS-CoV-2 and other coronaviruses in Southeast Asia. Experimental infections to test host range for this pathogen. Animal host(s) and any evidence of continued spill-over to humans. Socioeconomic and behavioral risk factors for this spill-over. Sustainable risk reduction strategies"
task_4 = "What do we know about vaccines and therapeutics? What has been published concerning research and development and evaluation efforts of vaccines and therapeutics? Effectiveness of drugs being developed and tried to treat COVID-19 patients. Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocycline that that may exert effects on viral replication. Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients. Exploration of use of best animal models and their predictive value for a human vaccine. Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents. Alternative models to aid decision makers in determining how to prioritize and distribute scarce, newly proven therapeutics as production ramps up. This could include identifying approaches for expanding production capacity to ensure equitable and timely distribution to populations in need. Efforts targeted at a universal coronavirus vaccine. Efforts to develop animal models and standardize challenge studies. Efforts to develop prophylaxis clinical studies and prioritize in healthcare workers. Approaches to evaluate risk for enhanced disease after vaccination. Assays to evaluate vaccine immune response and process development for vaccines, alongside suitable animal models in conjunction with therapeutics."
task_5 = "What has been published about medical care? What has been published concerning surge capacity and nursing homes? What has been published concerning efforts to inform allocation of scarce resources? What do we know about personal protective equipment? What has been published concerning alternative methods to advise on disease management? What has been published concerning processes of care? What do we know about the clinical characterization and management of the virus? Resources to support skilled nursing facilities and long term care facilities. Mobilization of surge medical staff to address shortages in overwhelmed communities. Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies. Extracorporeal membrane oxygenation (ECMO) outcomes data of COVID-19 patients. Outcomes data for COVID-19 after mechanical ventilation adjusted for age. Application of regulatory standards (e.g., EUA, CLIA) and ability to adapt care to crisis standards of care level. Approaches for encouraging and facilitating the production of elastomeric respirators, which can save thousands of N95 masks. Best telemedicine practices, barriers and facilitators, and specific actions to remove/expand them within and across state boundaries. Guidance on the simple things people can do at home to take care of sick people and manage disease. Oral medications that might potentially work. Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually. Best practices and critical challenges and innovative solutions and technologies in hospital flow and organization, workforce protection, workforce allocation, community-based support resources, payment, and supply chain management to enhance capacity, efficiency, and outcomes. Efforts to define the natural history of disease to inform clinical care, public health interventions, infection prevention control, transmission, and clinical trials. Efforts to develop a core clinical outcome set to maximize usability of data across a range of trials. Efforts to determine adjunctive and supportive interventions that can improve the clinical outcomes of infected patients (e.g. steroids, high flow oxygen). "
task_6 = "What do we know about the effectiveness of non-pharmaceutical interventions? What is known about equity and barriers to compliance for non-pharmaceutical interventions? Guidance on ways to scale up NPIs in a more coordinated way (e.g., establish funding, infrastructure and authorities to support real time, authoritative (qualified participants) collaboration with all states to gain consensus on consistent guidance and to mobilize resources to geographic areas where critical shortfalls are identified) to give us time to enhance our health care delivery system capacity to respond to an increase in cases. Rapid design and execution of experiments to examine and compare NPIs currently being implemented. DHS Centers for Excellence could potentially be leveraged to conduct these experiments. Rapid assessment of the likely efficacy of school closures, travel bans, bans on mass gatherings of various sizes, and other social distancing approaches. Methods to control the spread in communities, barriers to compliance and how these vary among different populations. Models of potential interventions to predict costs and benefits that take account of such factors as race, income, disability, age, geographic location, immigration status, housing status, employment status, and health insurance status. Policy changes necessary to enable the compliance of individuals with limited resources and the underserved with NPIs. Research on why people fail to comply with public health advice, even if they want to do so (e.g., social or financial costs may be too high). Research on the economic impact of this or any pandemic. This would include identifying policy and programmatic alternatives that lessen/mitigate risks to critical government services, food distribution and supplies, access to critical household supplies, and access to health diagnoses, treatment, and needed care, regardless of ability to pay."
task_7 = "What do we know about diagnostics and surveillance? What has been published concerning systematic, holistic approach to diagnostics (from the public health surveillance perspective to being able to predict clinical outcomes)? How widespread current exposure is to be able to make immediate policy recommendations on mitigation measures. Denominators for testing and a mechanism for rapidly sharing that information, including demographics, to the extent possible. Sampling methods to determine asymptomatic disease (e.g., use of serosurveys (such as convalescent samples) and early detection of disease (e.g., use of screening of neutralizing antibodies such as ELISAs). Efforts to increase capacity on existing diagnostic platforms and tap into existing surveillance platforms. Recruitment, support, and coordination of local expertise and capacity (public, private—commercial, and non-profit, including academic), including legal, ethical, communications, and operational issues. National guidance and guidelines about best practices to states (e.g., how states might leverage universities and private laboratories for testing purposes, communications to public health officials and the public). Development of a point-of-care test (like a rapid influenza test) and rapid bed-side tests, recognizing the tradeoffs between speed, accessibility, and accuracy. Rapid design and execution of targeted surveillance experiments calling for all potential testers using PCR in a defined area to start testing and report to a specific entity. These experiments could aid in collecting longitudinal samples, which are critical to understanding the impact of ad hoc local interventions (which also need to be recorded). Separation of assay development issues from instruments, and the role of the private sector to help quickly migrate assays onto those devices. Efforts to track the evolution of the virus (i.e., genetic drift or mutations) and avoid locking into specific reagents and surveillance/detection schemes. Latency issues and when there is sufficient viral load to detect the pathogen, and understanding of what is needed in terms of biological and environmental sampling. Use of diagnostics such as host response markers (e.g., cytokines) to detect early disease or predict severe disease progression, which would be important to understanding best clinical practice and efficacy of therapeutic interventions. Policies and protocols for screening and testing. Policies to mitigate the effects on supplies associated with mass testing, including swabs and reagents. Technology roadmap for diagnostics. Barriers to developing and scaling up new diagnostic tests (e.g., market forces), how future coalition and accelerator models (e.g., Coalition for Epidemic Preparedness Innovations) could provide critical funding for diagnostics, and opportunities for a streamlined regulatory environment. New platforms and technology (e.g., CRISPR) to improve response times and employ more holistic approaches to COVID-19 and future diseases. Coupling genomics and diagnostic testing on a large scale. Enhance capabilities for rapid sequencing and bioinformatics to target regions of the genome that will allow specificity for a particular variant. Enhance capacity (people, technology, data) for sequencing with advanced analytics for unknown pathogens, and explore capabilities for distinguishing naturally-occurring pathogens from intentional. One Health surveillance of humans and potential sources of future spillover or ongoing exposure for this organism and future pathogens, including both evolutionary hosts (e.g., bats) and transmission hosts (e.g., heavily trafficked and farmed wildlife and domestic food and companion species), inclusive of environmental, demographic, and occupational risk factors."
task_8 = "At the time of writing, COVID-19 has spread to at least 114 countries. With viral flu, there are often geographic variations in how the disease will spread and if there are different variations of the virus in different areas. We’d like to explore what the literature and data say about this through this Task. Are there geographic variations in the rate of COVID-19 spread? Are there geographic variations in the mortality rate of COVID-19? Is there any evidence to suggest geographic based virus mutations?"
task_9 = "What has been published concerning ethical considerations for research? What has been published concerning social sciences at the outbreak response? Efforts to articulate and translate existing ethical principles and standards to salient issues in COVID-2019. Efforts to embed ethics across all thematic areas, engage with novel ethical issues that arise and coordinate to minimize duplication of oversight. Efforts to support sustained education, access, and capacity building in the area of ethics. Efforts to establish a team at WHO that will be integrated within multidisciplinary research and operational platforms and that will connect with existing and expanded global networks of social sciences. Efforts to develop qualitative assessment frameworks to systematically collect information related to local barriers and enablers for the uptake and adherence to public health measures for prevention and control. This includes the rapid identification of the secondary impacts of these measures. (e.g. use of surgical masks, modification of health seeking behaviors for SRH, school closures). Efforts to identify how the burden of responding to the outbreak and implementing public health measures affects the physical and psychological health of those providing care for Covid-19 patients and identify the immediate needs that must be addressed. Efforts to identify the underlying drivers of fear, anxiety and stigma that fuel misinformation and rumor, particularly through social media."
task_10 = "What has been published about information sharing and inter-sectoral collaboration? What has been published about data standards and nomenclature? What has been published about governmental public health? What do we know about risk communication? What has been published about communicating with high-risk populations? What has been published to clarify community measures? What has been published about equity considerations and problems of inequity? Methods for coordinating data-gathering with standardized nomenclature. Sharing response information among planners, providers, and others. Understanding and mitigating barriers to information-sharing. How to recruit, support, and coordinate local (non-Federal) expertise and capacity relevant to public health emergency response (public, private, commercial and non-profit, including academic). Integration of federal/state/local public health surveillance systems. Value of investments in baseline public health response infrastructure preparedness. Modes of communicating with target high-risk populations (elderly, health care workers). Risk communication and guidelines that are easy to understand and follow (include targeting at risk populations’ families too). Communication that indicates potential risk of disease to all population groups. Misunderstanding around containment and mitigation. Action plan to mitigate gaps and problems of inequity in the Nation’s public health capability, capacity, and funding to ensure all citizens in need are supported and can access information, surveillance, and treatment. Measures to reach marginalized and disadvantaged populations. Data systems and research priorities and agendas incorporate attention to the needs and circumstances of disadvantaged populations and underrepresented minorities. Mitigating threats to incarcerated people from COVID-19, assuring access to information, prevention, diagnosis, and treatment. Understanding coverage policies (barriers and opportunities) related to testing, treatment, and care."


list_of_tasks = [task_1, task_2, task_3, task_4, task_5, task_6, task_7, task_8, task_9, task_10]
list_of_tasks_short = [task_1_short, task_2_short, task_3_short, task_4_short, task_5_short, task_6_short, task_7_short, task_8_short, task_9_short, task_10_short]

In [264]:
# helper files

from sklearn.metrics import ndcg_score
import numpy as np

#load the qrels file
def getqrels(fname):
    qrels = []
    for line in open(fname).readlines():
        vals = line.strip().split(" ")
        #topic, cord_uid, qrel, assessround
        qrels.append([int(vals[0]), vals[3], float(vals[4]), float(vals[1])])

    qrels = np.array(qrels, dtype="O")
    return qrels

#find qrel for a cord uid
def get_qrel(cord_uid, topic_id, qrels):

    topicrels = qrels[qrels[:,0] == topic_id]

    qrel_uids = [qrel[1] for qrel in topicrels]
    #print(qrel_uids.index(cord_uid))
    index = qrel_uids.index(cord_uid)
    #print(qrels[index,2])
    if(qrels[index,2] > 0):
        return 1
    else:
        return 0


#create a list of length N with K ones and N-K zeroes
#was used for old version of kfold xvalidation
def rand_bin_list(K, N):
    arr = np.zeros(N)
    arr[:K]  = 1
    np.random.shuffle(arr)
    return list(arr)

    

#ndcg after filtering unknown docs - sakai 2007 says this is more stable than bpref
def ndcg(runname, qrelname):
    qrels = getqrels(qrelname)
#    print(np.array(qrels).shape)
    
    preds = []
    for line in open(runname).readlines():
        #topic, unused, cord_uid, rank, score, runname
        vals = line.strip().split(" ")
        #topic, cord_uid, score, rank
        preds.append([int(vals[0]), vals[2], float(vals[4]), int(vals[3])])
        
#    print(np.array(preds).shape)

    knownpreds = []
    
    for num, pred in enumerate(preds):
        #get qrels for the given topic
        qrels_topic = qrels[qrels[:,0] == pred[0]]
        qrel_uids = [val[1] for val in qrels_topic]
    
        #filter all preds not in qrels
        if(pred[1] in qrel_uids):
            #add real val
            knownpreds.append([pred[0], pred[1], pred[2], get_qrel(pred[1], pred[0], qrels_topic), pred[3]])
            #print(str(pred[0]) + ' '+ str(pred[1]))

    #prep data so we have
    #true score, pred score
    #print('so how many do we know')
    #print(len(knownpreds))
    #print(len(preds))
    #print(knownpreds[0:5])
    #print(preds[0:5])

    knownpreds = np.array(knownpreds, dtype="O")
    #print(knownpreds.shape)

        
    #print(sortedqrel[0:20])
    #print(sortedqpred[0:20])
    
    #TODO for each topic compute ndcg, then average
    ndcgs = []
    for t in range(1, 31):
        #print('known preds')
        #print(knownpreds)
        #print(knownpreds[:,0])
        #get this topic's results after filtering unknown qrels
        
        
        
        #print('here')
        #print(t)
        #print(np.array(knownpreds).shape)
        #print(knownpreds)
        knownpreds_t = knownpreds[knownpreds[:,0] == t]
        #anchor2

        
        #cross validation on knownpreds_t
        n_splits = 5
        if(len(knownpreds_t) > 5):
    #        n_splits = 1
            kf = KFold(n_splits)
            for train_index, test_index in kf.split(knownpreds_t):
            #print(train_index)
#        for i in range(0, 5):
            #if 1, ignore from this run
#            crossval_ind = rand_bin_list(np.floor(len(knownpreds_ts) / 5), len(knownpreds_ts))
#            print(crossval_ind)
            

        
                sortedqrel = []
                sortedqpred = []
                for pred_ind, pred in enumerate(knownpreds_t):
                    if pred_ind in train_index:
                #if crossval_ind[pred_ind] == 0:
                    #print('rankingnew task')
                    #print(pred[1])
                    #print(pred[0])
                    #print(get_qrel(pred[1], pred[0], qrels))
                    #get corresponding pred's qrel
                        sortedqrel.append(pred[3])#get_qrel(pred[1], pred[0], qrels))
                    #ground truth: is supposed to be a '2'
                        sortedqpred.append(pred[2])
            
                #print('topic ' + str(t) + ' has qrels ' + str(len(sortedqrel)))#' knownpreds')
                #print(knownpreds_t[0:5])
                #print(sortedqrel[0:5])
                #print(sortedqpred[0:5])
        
        
        
            
                if(len(sortedqrel) > 1):
                        #print(np.array([sortedqrel]))
                        #print(np.array([sortedqpred]))
        #            print(ndcg_score(np.asarray([sortedqrel]), np.asarray([sortedqpred]), k=5))
                    ndcgs.append(ndcg_score(np.asarray([sortedqrel]), np.asarray([sortedqpred]), k=10))
                else:
                    pass
                    #print('huh')
           
        #copypasted from above.. should fix later
        else:
            pass
#            sortedqrel = []
#            sortedqpred = []
#            for pred in knownpreds_t:
#                sortedqrel.append(pred[3])
#                sortedqpred.append(pred[2])
            
#                if(len(sortedqrel) > 1):
#                        ndcgs.append(ndcg_score(np.asarray([sortedqrel]), np.asarray([sortedqpred]), k=10))
#                else:
#                    pass
                    #print('huh')
           
        
        
    #print(ndcgs)
    return np.mean(ndcgs)

In [265]:
bm25_split = .60
nar_split = .25
task_split = .15


#should we filter old qrels etc for round 3?  i.e. False = generate using valid doc ids etc
unfiltered = True


#todo update xval by always ranking full qrels   anchor3 
#for i in range(0, len(docids)):
#    score = index_utils.compute_query_document_score(docids[i], query)
#    print(f'{i+1:2} {docids[i]:15} {score:.5f}')

def testrun2(fname, pq, pnarr, ptask):

    qrel_seen = 0
    total = 0

    with open(fname, 'w') as f:
        for topicno, topic in tqdm(enumerate(topics)):
                task = tasks[rnd3classes[topicno]]
                topic = topic.split(' ')
                nar = nars[topicno]
                nar = nar.split(' ')    
                new_task = []
                for t in task:
                    for nt in t.split(' '):
                        new_task.append(nt)
                #use with kaggle task text
                task = new_task
                #task_confidence = 1
                
                #use with qrel task text
                #task = tasks_weights[topic_task_no[topicno]]
                task_confidence = rnd3confidence[rnd3classes[topicno]]
                
                query = build_query(topic, nar, task, pq, pnarr, ptask, task_confidence, rnd3classes[topicno])
                
                #for ranking qrels
                #mast
                
                #topic, cord, rel, round
#                qrels = getqrels(qrelname)
#                hits = []
                
#                print(query)
#                for qrel in qrels:
#                    score = index_utils.compute_query_document_score(qrel[1], query)
#                    hits.append([qrel].extend(score))
#                    print(hits)
                    
#                rankedhits = sorted(hits, key=lambda x: x[4])
                
#                print(hits[0:5])
#                print(rankedhits[0:5])
#                print()
                
                
                hits = searcher.search(query, 20000) #5000 for filtering

                i = 0
                j = 0
                seen = set()
                

                #for regular ranking
                while j < len(hits):#i < 100:   #for generating the 100 end results
                    hit = hits[j]
                    # print(hit)
                    if(unfiltered):
                        #pass
                        if(hit.docid.split('.')[0] not in judged[topicno+1]) or hit.docid.split('.')[0] in seen:
                            j+=1
                            continue
                    else:
                        if hit.docid.split('.')[0] in seen or hit.docid.split('.')[0] in judged[topicno+1] or hit.docid.split('.')[0] not in valid:
                            j+=1
                            continue
                    f.write(f'{topicno+1} Q0 {hit.docid.split(".")[0]} {i+1} {hit.score} {fname[42:-4]}\n')
                    i+=1
                    j+=1
                    seen |= {hit.docid.split('.')[0]}

                    if hit.docid.split('.')[0] in judged[topicno+1]:
                        qrel_seen +=1
                    total+=1


qrel_path = '/home/tmschoegje/Desktop/caos-19/trecdata/qrels-rnd2.txt'
topics_path = '/home/tmschoegje/Desktop/caos-19/trecdata/topics-rnd2.xml'         
tree = Et.parse(topics_path)
root = tree.getroot()
topics = [root[i][0].text for i in range(35)]                    
nars =  [root[i][1].text for i in range(35)]                    
        
judged = defaultdict(set)
with open(qrel_path, 'r') as f:
    for line in f:
        topicno, iteration, empty, docid, relevance = line.strip().split(' ')
        judged[int(topicno)] |= {docid}
    
pq = 0.6
pnarr = 0.25
ptask = 0.15
qrelname = '/home/tmschoegje/Desktop/caos-19/trecdata/qrels-rnd2.txt'

lognames = []
logscores = []

#generate baseline run - only query
#runname = '/home/tmschoegje/Desktop/caos-19/runs/testrun-baseline-rnd3.run'
#testrun2(runname, 1, 0, 0)
#print(ndcg(runname, qrelname))

#generate using the best known parameters
#runname = '/home/tmschoegje/Desktop/caos-19/runs/testrun-best-rnd3.run'
#testrun2(runname, 0.2, 0.4, 0.4)
#print(ndcg(runname, qrelname))


for pq in np.linspace(0, 1, 3):
    for pnarr in np.linspace(0, 1, 3):
        for ptask in np.linspace(0, 1, 3):
            ptotal = pq+pnarr+ptask
            
            if (pq == 0):
                pq_n = 0
            else:
                pq_n = pq / ptotal
                
            if (pnarr == 0):
                pnarr_n = 0
            else:
                pnarr_n = pnarr / ptotal
                
            if (ptask == 0):
                ptask_n = 0
            else:
                ptask_n = ptask / ptotal
            runname = '/home/tmschoegje/Desktop/caos-19/trecdata/qexp-q' + str(pq_n) + 'n' + str(pnarr_n) + 't' + str(ptask_n) + '.run'
            testrun2(runname, pq, pnarr, ptask)
            n = ndcg(runname, qrelname)
            print('Run ' + str(runname) + ' ndcg ' + str(n))
            
            lognames.append(str(runname))
            logscores.append(n)


#runname = '/home/tmschoegje/Desktop/caos-19/trecdata/qexp-test1'
#testrun(runname, 1, 0, 0)
#print(ndcg(runname, qrelname))

"""
m = max(logscores)
print('max score found was')
print(m)
print('which was found in runs')
index = [i for i, j in enumerate(logscores) if j == m]
for i in index:
    print(lognames[i])
    
#best so far - 2/3 query + 1/3 narr    .55
"""

#so doing the tasks qexp gives a max of .578
#doing the query qexp gives          Run /home/tmschoegje/Desktop/caos-19/trecdata/qexp-q0.25n0.5t0.25.run ndcg 0.6041186304778956
#doing top3 narrative term selection based on embedding gives
#doing top3 narrative term selection based on tfidf gives

15it [00:38,  2.46s/it]

35it [01:29,  2.55s/it]
0it [00:00, ?it/s]

Run /home/tmschoegje/Desktop/caos-19/trecdata/qexp-q0n0t0.run ndcg 0.5950365508357838


15it [00:32,  2.11s/it]

35it [01:14,  2.14s/it]
0it [00:00, ?it/s]

...

Run /home/tmschoegje/Desktop/caos-19/trecdata/qexp-q0.4n0.4t0.2.run ndcg 0.5757659409248462


15it [00:32,  2.21s/it]

35it [01:18,  2.23s/it]


Run /home/tmschoegje/Desktop/caos-19/trecdata/qexp-q0.3333333333333333n0.3333333333333333t0.3333333333333333.run ndcg 0.5723061318613217


"\nm = max(logscores)\nprint('max score found was')\nprint(m)\nprint('which was found in runs')\nindex = [i for i, j in enumerate(logscores) if j == m]\nfor i in index:\n    print(lognames[i])\n    \n#best so far - 2/3 query + 1/3 narr    .55\n"

In [ ]:
bm25 = ['coronavirus', 'response', 'weather', 'changes', 'Coronavirus', 'CoV', 'coronaviruses', 'responses', 'defenses', 'suppression', 'they', 'thus', 'climatic', 'meteorological', 'climate', 'alterations', 'change', 'differences']
builder = pyquerybuilder.get_boolean_query_builder()
    
for b in bm25:
    b = pyquerybuilder.get_term_query(b)
    boostquery = pyquerybuilder.get_boost_query(b, bm25_split/len(bm25))
    builder.add(boostquery, pyquerybuilder.JBooleanClauseOccur['should'].value)
